In [1]:
# import the libraries
import numpy as np
import pandas as pd
import re 
from Bio import SeqIO
from collections import defaultdict

In [2]:
# Create a dataframe for Bandage Combined1
master_df= pd.read_csv('QueriesAndResultsMasterList.tsv', sep='\t')

In [3]:
master_df

,Unnamed: 0,Query,Sequence_Query,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Sequence_Bandage,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Path_GraphAligner,Start_GraphAligner,End_GraphAligner,BandageVSSPAligner
0,0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17.0,877.0,False
1,1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,"(47) 3478183-, 50639855- (37) , (47) 3478183-,...","47, 47","37, 37","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,852,806,861,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,907.0,False
2,2,gb|AB200915.1|-|1831-2305|ARO:3005084|dfrA31,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,474,(8) 54829473- (481),8,481,54829473,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,7,426,419,474,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,54829473,8.0,481.0,False
3,3,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17.0,877.0,False
4,4,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17.0,877.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,684,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,904.0,False
685,685,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,904.0,False
686,686,gb|Y17584|+|0-858|ARO:3000899|TEM-29,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,904.0,False
687,687,gb|Z22590|+|154-955|ARO:3001406|OXA-11,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,801,(660) 104604+ (1460),660,1460,104604,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,659,1405,746,801,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,104604,660.0,1460.0,False


In [4]:
#Drop the columns
master_df=master_df.drop(['BandageVSSPAligner','SPAlignerVSGraphAligner'],axis=1)

KeyError: "['SPAlignerVSGraphAligner'] not found in axis"

In [5]:
master_df

,Unnamed: 0,Query,Sequence_Query,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Sequence_Bandage,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Path_GraphAligner,Start_GraphAligner,End_GraphAligner,BandageVSSPAligner
0,0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17.0,877.0,False
1,1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,"(47) 3478183-, 50639855- (37) , (47) 3478183-,...","47, 47","37, 37","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,852,806,861,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,907.0,False
2,2,gb|AB200915.1|-|1831-2305|ARO:3005084|dfrA31,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,474,(8) 54829473- (481),8,481,54829473,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,7,426,419,474,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,54829473,8.0,481.0,False
3,3,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17.0,877.0,False
4,4,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17.0,877.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,684,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,904.0,False
685,685,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,904.0,False
686,686,gb|Y17584|+|0-858|ARO:3000899|TEM-29,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,904.0,False
687,687,gb|Z22590|+|154-955|ARO:3001406|OXA-11,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,801,(660) 104604+ (1460),660,1460,104604,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,659,1405,746,801,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,104604,660.0,1460.0,False


In [6]:
#Remove the + or - from the Path_SPAligner
#master_df['Path_SPAligner']=master_df['Path_SPAligner'].str.replace(r'[+-]', '', regex=True)
master_df['End_Bandage']=master_df['End_Bandage'].str.replace(r'[+-]', '', regex=True)

In [7]:
#Add +2 to the values of Start_SPAligner to comapre with Start_GraphAligner
master_df['Start_SPAligner_GraphAligner'] = master_df['Start_SPAligner'].apply(lambda x: ', '.join(str(int(val) + 2) for val in x.split(',')))

In [8]:
#Add +1 to the values of Start_SPAligner to comapre with Start_GraphAligner
master_df['Start_SPAligner_Bandage'] = master_df['Start_SPAligner'].apply(lambda x: ', '.join(str(int(val) + 1) for val in x.split(',')))

In [9]:
master_df

,Unnamed: 0,Query,Sequence_Query,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Sequence_Bandage,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Path_GraphAligner,Start_GraphAligner,End_GraphAligner,BandageVSSPAligner,Start_SPAligner_GraphAligner,Start_SPAligner_Bandage
0,0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17.0,877.0,False,18,17
1,1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,"(47) 3478183-, 50639855- (37) , (47) 3478183-,...","47, 47","37, 37","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,852,806,861,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,907.0,False,48,47
2,2,gb|AB200915.1|-|1831-2305|ARO:3005084|dfrA31,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,474,(8) 54829473- (481),8,481,54829473,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,7,426,419,474,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,54829473,8.0,481.0,False,9,8
3,3,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17.0,877.0,False,18,17
4,4,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17.0,877.0,False,18,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,684,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,904.0,False,48,47
685,685,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,904.0,False,48,47
686,686,gb|Y17584|+|0-858|ARO:3000899|TEM-29,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,904.0,False,48,47
687,687,gb|Z22590|+|154-955|ARO:3001406|OXA-11,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,801,(660) 104604+ (1460),660,1460,104604,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,659,1405,746,801,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,104604,660.0,1460.0,False,661,660


In [10]:
#Remove the braces from the end_Bandage
master_df['End_Bandage'] = master_df['End_Bandage'].str.replace('(', '').str.replace(')', '')
master_df

,Unnamed: 0,Query,Sequence_Query,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Sequence_Bandage,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Path_GraphAligner,Start_GraphAligner,End_GraphAligner,BandageVSSPAligner,Start_SPAligner_GraphAligner,Start_SPAligner_Bandage
0,0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17.0,877.0,False,18,17
1,1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,"(47) 3478183-, 50639855- (37) , (47) 3478183-,...","47, 47","37, 37","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,852,806,861,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,907.0,False,48,47
2,2,gb|AB200915.1|-|1831-2305|ARO:3005084|dfrA31,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,474,(8) 54829473- (481),8,481,54829473,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,7,426,419,474,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,54829473,8.0,481.0,False,9,8
3,3,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17.0,877.0,False,18,17
4,4,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17.0,877.0,False,18,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,684,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,904.0,False,48,47
685,685,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,904.0,False,48,47
686,686,gb|Y17584|+|0-858|ARO:3000899|TEM-29,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47.0,904.0,False,48,47
687,687,gb|Z22590|+|154-955|ARO:3001406|OXA-11,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,801,(660) 104604+ (1460),660,1460,104604,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,659,1405,746,801,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,104604,660.0,1460.0,False,661,660


## This section compares the result from SPAligner and Bandage

In [11]:
#Convert Path_GraphAligner, Start_GraphAligner and End_GraphAligner into int
master_df= master_df.fillna(000)
master_df['Start_GraphAligner']=master_df['Start_GraphAligner'].astype(int)
master_df['End_GraphAligner']=master_df['End_GraphAligner'].astype(int)
#master_df['Path_GraphAligner']=master_df['Path_GraphAligner'].astype(int)
master_df

,Unnamed: 0,Query,Sequence_Query,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Sequence_Bandage,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Path_GraphAligner,Start_GraphAligner,End_GraphAligner,BandageVSSPAligner,Start_SPAligner_GraphAligner,Start_SPAligner_Bandage
0,0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17,877,False,18,17
1,1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,"(47) 3478183-, 50639855- (37) , (47) 3478183-,...","47, 47","37, 37","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,852,806,861,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47,907,False,48,47
2,2,gb|AB200915.1|-|1831-2305|ARO:3005084|dfrA31,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,474,(8) 54829473- (481),8,481,54829473,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,7,426,419,474,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,54829473,8,481,False,9,8
3,3,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17,877,False,18,17
4,4,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17,877,False,18,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,684,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47,904,False,48,47
685,685,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47,904,False,48,47
686,686,gb|Y17584|+|0-858|ARO:3000899|TEM-29,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47,904,False,48,47
687,687,gb|Z22590|+|154-955|ARO:3001406|OXA-11,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,801,(660) 104604+ (1460),660,1460,104604,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,659,1405,746,801,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,104604,660,1460,False,661,660


In [12]:
master_df.dtypes

Unnamed: 0                       int64
Query                           object
Sequence_Query                  object
Length_Bandage                   int64
Path_Bandage                    object
Start_Bandage                   object
End_Bandage                     object
Extracted_Path                  object
Sequence_Bandage                object
Start_SPAligner                 object
End_SPAligner                   object
Length_SPAligner                object
Path_SPAligner                   int64
Sequence_SPAligner              object
Path_GraphAligner               object
Start_GraphAligner               int64
End_GraphAligner                 int64
BandageVSSPAligner                bool
Start_SPAligner_GraphAligner    object
Start_SPAligner_Bandage         object
dtype: object

In [13]:
master_df['Start_SPAligner']=master_df['Start_SPAligner'].astype(str)
#master_df['Start_SPAligner_GraphAligner']=master_df['Start_SPAligner_GraphAligner'].astype(str)
#master_df['Start_SPAligner_Bandage']=master_df['Start_SPAligner_Bandage'].astype(str)

master_df['End_SPAligner']=master_df['End_SPAligner'].astype(str)
master_df['Path_SPAligner']=master_df['Path_SPAligner'].astype(str)

In [14]:
master_df

,Unnamed: 0,Query,Sequence_Query,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Sequence_Bandage,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Path_GraphAligner,Start_GraphAligner,End_GraphAligner,BandageVSSPAligner,Start_SPAligner_GraphAligner,Start_SPAligner_Bandage
0,0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17,877,False,18,17
1,1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,"(47) 3478183-, 50639855- (37) , (47) 3478183-,...","47, 47","37, 37","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,852,806,861,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47,907,False,48,47
2,2,gb|AB200915.1|-|1831-2305|ARO:3005084|dfrA31,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,474,(8) 54829473- (481),8,481,54829473,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,7,426,419,474,ATGAAAATATCCATTATGGCAGCAGTTTCTGAGAATGGAGTAATTG...,54829473,8,481,False,9,8
3,3,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17,877,False,18,17
4,4,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(17) 1612535+ (877),17,877,1612535,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,16,822,806,861,ATGCGTTATATTCGCCTGTGTATTATCACCCTGTGAGCCACCCTGC...,1612535,17,877,False,18,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,684,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47,904,False,48,47
685,685,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47,904,False,48,47
686,686,gb|Y17584|+|0-858|ARO:3000899|TEM-29,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,46,849,803,858,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,3478183,47,904,False,48,47
687,687,gb|Z22590|+|154-955|ARO:3001406|OXA-11,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,801,(660) 104604+ (1460),660,1460,104604,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,659,1405,746,801,ATGAAAACATTTGCCGCATATGTAATTATCGCGTGTCTTTCGAGTA...,104604,660,1460,False,661,660


In [15]:
def compare_values_Bandage_SPAligner(row):
    if row['Start_Bandage'] == row['Start_SPAligner'] and row['End_Bandage'] == row['End_SPAligner'] and row['Extracted_Path'] == row['Path_SPAligner']:
        return 'Full'
    elif row['Extracted_Path'] == row['Path_SPAligner']and row['Start_Bandage'] != row['Start_SPAligner'] and row['End_Bandage'] != row['End_SPAligner']:
        return 'MatchOnPath'
    elif row['Extracted_Path'] == row['Path_SPAligner'] and row['Start_Bandage'] == row['Start_SPAligner'] and row['End_Bandage'] != row['End_SPAligner'] :
        return 'MatchOnPathAndStart'
    elif row['Extracted_Path'] == row['Path_SPAligner'] and row['Start_Bandage'] != row['Start_SPAligner'] and row['End_Bandage'] == row['End_SPAligner'] :
        return 'MatchOnPathAndEnd'
    else:
        return 'Different'

In [16]:
master_df['BandageVSSPAligner'] = master_df.apply(compare_values_Bandage_SPAligner, axis=1)

In [17]:
master_df.dtypes

Unnamed: 0                       int64
Query                           object
Sequence_Query                  object
Length_Bandage                   int64
Path_Bandage                    object
Start_Bandage                   object
End_Bandage                     object
Extracted_Path                  object
Sequence_Bandage                object
Start_SPAligner                 object
End_SPAligner                   object
Length_SPAligner                object
Path_SPAligner                  object
Sequence_SPAligner              object
Path_GraphAligner               object
Start_GraphAligner               int64
End_GraphAligner                 int64
BandageVSSPAligner              object
Start_SPAligner_GraphAligner    object
Start_SPAligner_Bandage         object
dtype: object

In [18]:
master_df.to_csv("TestCompare1.tsv",sep='\t')

In [19]:
BandagevsSPAligner_df=master_df[['Query','Path_Bandage','Start_Bandage','End_Bandage','Extracted_Path',
                                'Start_SPAligner','End_SPAligner','Path_SPAligner']]

In [20]:
BandagevsSPAligner_df

,Query,Path_Bandage,Start_Bandage,End_Bandage,Extracted_Path,Start_SPAligner,End_SPAligner,Path_SPAligner
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,(17) 1612535+ (877),17,877,1612535,16,822,861
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,"(47) 3478183-, 50639855- (37) , (47) 3478183-,...","47, 47","37, 37","3478183, 50639855,3478183, 36393793",46,852,861
2,gb|AB200915.1|-|1831-2305|ARO:3005084|dfrA31,(8) 54829473- (481),8,481,54829473,7,426,474
3,gb|AB302939|+|8-869|ARO:3001115|SHV-60,(17) 1612535+ (877),17,877,1612535,16,822,861
4,gb|AB372881|+|8-869|ARO:3001160|SHV-111,(17) 1612535+ (877),17,877,1612535,16,822,861
...,...,...,...,...,...,...,...,...
684,gb|Y17582|+|0-858|ARO:3000892|TEM-21,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",46,849,858
685,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",46,849,858
686,gb|Y17584|+|0-858|ARO:3000899|TEM-29,"(47) 3478183-, 50639855- (34) , (47) 3478183-,...","47, 47","34, 34","3478183, 50639855,3478183, 36393793",46,849,858
687,gb|Z22590|+|154-955|ARO:3001406|OXA-11,(660) 104604+ (1460),660,1460,104604,659,1405,801


In [21]:
BandagevsSPAligner_df['BandageVSSPAligner'] = BandagevsSPAligner_df.apply(compare_values_Bandage_SPAligner, axis=1)

/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_1412/1140914095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsSPAligner_df['BandageVSSPAligner'] = BandagevsSPAligner_df.apply(compare_values_Bandage_SPAligner, axis=1)


In [22]:
BandagevsSPAligner_df.to_csv('TestRes.tsv',sep='\t')

In [23]:
BandagevsSPAligner_df.dtypes

Query                 object
Path_Bandage          object
Start_Bandage         object
End_Bandage           object
Extracted_Path        object
Start_SPAligner       object
End_SPAligner         object
Path_SPAligner        object
BandageVSSPAligner    object
dtype: object

In [24]:
output_list = []

for index, row in BandagevsSPAligner_df.iterrows():
    path_values = str(row['Extracted_Path']).split(',')
    start_values = str(row['Start_Bandage']).split(',')
    end_values = str(row['End_Bandage']).split(',')
    
    path_value = row['Path_SPAligner']
    start_value= str(row['Start_SPAligner']).split(',')
    end_value= str(row['End_SPAligner']).split(',')
    
    if path_value in path_values  and start_value in start_values and end_value in end_values:
        output_list.append('SinglePathFull')
    elif path_value in path_values and start_value in start_values and end_value not in end_values:
        output_list.append('SinglePathStartMatch')
    elif path_value in path_values and start_value not in start_values and end_value in end_values:
        output_list.append('SinglePathEndMatch')
    elif path_value in path_values and start_value not in start_values and end_value not in end_values:
        output_list.append('SinglePathMatch')
    elif path_value not in path_values and start_value in start_values and end_value not in end_values:
        output_list.append('SingleStartMatch')
    else:
         output_list.append('Different')

BandagevsSPAligner_df['ResultsBandageVSSPAligner'] = output_list

/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_1412/3651665796.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsSPAligner_df['ResultsBandageVSSPAligner'] = output_list


In [25]:
def determine_final_result(row):
    if row['BandageVSSPAligner'] == 'Full':
        return 'Full'
    elif row['BandageVSSPAligner'] == 'MatchOnPath':
        return 'MatchOnPath'
    else:
        return row['ResultsBandageVSSPAligner']

In [26]:
BandagevsSPAligner_df.dtypes

Query                        object
Path_Bandage                 object
Start_Bandage                object
End_Bandage                  object
Extracted_Path               object
Start_SPAligner              object
End_SPAligner                object
Path_SPAligner               object
BandageVSSPAligner           object
ResultsBandageVSSPAligner    object
dtype: object

In [27]:
BandagevsSPAligner_df['FinalResultBandageVSSPAligner'] = BandagevsSPAligner_df.apply(determine_final_result, axis=1)

/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_1412/761307203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsSPAligner_df['FinalResultBandageVSSPAligner'] = BandagevsSPAligner_df.apply(determine_final_result, axis=1)


In [28]:
BandagevsSPAligner_df.to_csv('TestResultCategoriesBandageSPAligner.tsv',sep='\t')

## This section compares the values from GraphAligner and Bandage

In [29]:
master_df['Start_GraphAligner']=master_df['Start_GraphAligner'].astype(str)
master_df['End_GraphAligner']=master_df['End_GraphAligner'].astype(str)
master_df['Path_GraphAligner']=master_df['Path_GraphAligner'].astype(str)

In [30]:
BandagevsGraphAligner_df=master_df[['Query','Path_Bandage','Start_Bandage','End_Bandage','Extracted_Path',
                                'Start_GraphAligner','End_GraphAligner','Path_GraphAligner']]

In [31]:
BandagevsGraphAligner_df.dtypes


Query                 object
Path_Bandage          object
Start_Bandage         object
End_Bandage           object
Extracted_Path        object
Start_GraphAligner    object
End_GraphAligner      object
Path_GraphAligner     object
dtype: object

In [32]:
def compare_values_Bandage_GraphAligner(row):
    if row['Start_Bandage'] == row['Start_GraphAligner'] and row['End_Bandage'] == row['End_GraphAligner'] and row['Extracted_Path'] == row['Path_GraphAligner']:
        return 'Full'
    elif row['Extracted_Path'] == row['Path_GraphAligner']  and row['Start_Bandage'] != row['Start_GraphAligner'] and row['End_Bandage'] != row['End_GraphAligner'] :
        return 'MatchOnPath'
    elif row['Extracted_Path'] == row['Path_GraphAligner'] and row['Start_Bandage'] == row['Start_GraphAligner'] and row['End_Bandage'] != row['End_GraphAligner'] :
        return 'MatchOnPathAndStart'
    elif row['Extracted_Path'] == row['Path_GraphAligner'] and row['Start_Bandage'] != row['Start_GraphAligner'] and row['End_Bandage'] == row['End_GraphAligner'] :
        return 'MatchOnPathAndEnd'
    else:
        return 'Different'

In [33]:
BandagevsGraphAligner_df['BandageVSGraphAligner'] = BandagevsGraphAligner_df.apply(compare_values_Bandage_GraphAligner, axis=1)

/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_1412/607607642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsGraphAligner_df['BandageVSGraphAligner'] = BandagevsGraphAligner_df.apply(compare_values_Bandage_GraphAligner, axis=1)


In [34]:
BandagevsGraphAligner_df.to_csv('rESULT44.tsv',sep='\t')

In [35]:
output_list1 = []

for index, row in BandagevsGraphAligner_df.iterrows():
    path_values = str(row['Extracted_Path']).split(',')
    start_values = str(row['Start_Bandage']).split(',')
    end_values = str(row['End_Bandage']).split(',')
    
    path_value = str(row['Path_GraphAligner']).split(',')
    start_value= str(row['Start_GraphAligner']).split(',')
    end_value= str(row['End_GraphAligner']).split(',')
    
    if path_value in path_values and start_value in start_values and end_value in end_values:
        output_list1.append('SinglePathFull')
    elif path_value in path_values and start_value in start_values and end_value not in end_values:
        output_list1.append('SinglePathStartMatch')
    elif path_value in path_values and start_value not in start_values and end_value  in end_values:
        output_list1.append('SinglePathEndMatch')
    elif path_value in path_values and start_value not in start_values and end_value not in end_values:
        output_list1.append('SinglePathMatch')
    else:
         output_list1.append('Different')
    

BandagevsGraphAligner_df['ResultsBandageVSGraphAligner'] = output_list1

/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_1412/2602096735.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsGraphAligner_df['ResultsBandageVSGraphAligner'] = output_list1


In [36]:
BandagevsGraphAligner_df.to_csv('rESULT445.tsv',sep='\t')

In [37]:
def determine_final_result_1(row):
    if row['BandageVSGraphAligner'] == 'Full':
        return 'Full'
    elif row['BandageVSGraphAligner'] == 'MatchOnPath':
        return 'MatchOnPath'
    else:
        return row['ResultsBandageVSGraphAligner']

In [38]:
BandagevsGraphAligner_df['FinalResultBandageVSGraphAligner'] = BandagevsGraphAligner_df.apply(determine_final_result_1, axis=1)

/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_1412/2292256490.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BandagevsGraphAligner_df['FinalResultBandageVSGraphAligner'] = BandagevsGraphAligner_df.apply(determine_final_result_1, axis=1)


In [39]:
BandagevsGraphAligner_df.to_csv('TestResultCategoriesBandageGraphAligner.tsv',sep='\t')

## This section compares the results from GraphAligner and SPAligner

In [40]:
SPAlignervsGraphAligner_df=master_df[['Query','Start_SPAligner','End_SPAligner','Path_SPAligner',
                                'Start_GraphAligner','End_GraphAligner','Path_GraphAligner','Start_SPAligner_GraphAligner']]

In [41]:
SPAlignervsGraphAligner_df.dtypes

Query                           object
Start_SPAligner                 object
End_SPAligner                   object
Path_SPAligner                  object
Start_GraphAligner              object
End_GraphAligner                object
Path_GraphAligner               object
Start_SPAligner_GraphAligner    object
dtype: object

In [42]:
def compare_values_GraphAligner_SPAligner(row):
    if row['Start_SPAligner_GraphAligner'] == row['Start_GraphAligner'] and row['End_SPAligner'] == row['End_GraphAligner'] and row['Path_SPAligner'] == row['Path_GraphAligner']:
        return 'Full'
    elif row['Path_SPAligner'] == row['Path_GraphAligner']and row['Start_SPAligner_GraphAligner'] != row['Start_GraphAligner']and row['End_SPAligner'] == row['End_GraphAligner'] :
        return 'MatchOnPathAndEnd'
    elif row['Path_SPAligner'] == row['Path_GraphAligner'] and row['Start_SPAligner_GraphAligner'] == row['Start_GraphAligner'] and row['End_SPAligner'] != row['End_GraphAligner'] :
        return 'MatchOnPathAndStart'
    elif row['Path_SPAligner'] == row['Path_GraphAligner'] and row['Start_SPAligner_GraphAligner'] != row['Start_GraphAligner'] and row['End_SPAligner'] != row['End_GraphAligner'] :
        return 'MatchOnPath'
    else:
        return 'Different'

In [43]:
SPAlignervsGraphAligner_df['SPAlignerVSGraphAligner'] = SPAlignervsGraphAligner_df.apply(compare_values_GraphAligner_SPAligner, axis=1)

/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_1412/33428813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SPAlignervsGraphAligner_df['SPAlignerVSGraphAligner'] = SPAlignervsGraphAligner_df.apply(compare_values_GraphAligner_SPAligner, axis=1)


In [44]:
SPAlignervsGraphAligner_df.to_csv('TestResultCategoriesGraphAlignerSPAligner.tsv',sep='\t')

In [45]:


# Assuming your three data frames are named df1, df2, and df3
# Merge df1 and df2 based on the common column "Query"
merged_df_cat = pd.merge(BandagevsSPAligner_df, BandagevsGraphAligner_df, on='Query', how='inner')

# Merge the result with df3 based on the common column "Query"
final_df = pd.merge(merged_df_cat, SPAlignervsGraphAligner_df, on='Query', how='inner')


In [46]:
final_df.columns


Index(['Query', 'Path_Bandage_x', 'Start_Bandage_x', 'End_Bandage_x',
       'Extracted_Path_x', 'Start_SPAligner_x', 'End_SPAligner_x',
       'Path_SPAligner_x', 'BandageVSSPAligner', 'ResultsBandageVSSPAligner',
       'FinalResultBandageVSSPAligner', 'Path_Bandage_y', 'Start_Bandage_y',
       'End_Bandage_y', 'Extracted_Path_y', 'Start_GraphAligner_x',
       'End_GraphAligner_x', 'Path_GraphAligner_x', 'BandageVSGraphAligner',
       'ResultsBandageVSGraphAligner', 'FinalResultBandageVSGraphAligner',
       'Start_SPAligner_y', 'End_SPAligner_y', 'Path_SPAligner_y',
       'Start_GraphAligner_y', 'End_GraphAligner_y', 'Path_GraphAligner_y',
       'Start_SPAligner_GraphAligner', 'SPAlignerVSGraphAligner'],
      dtype='object')

In [47]:
columns_to_drop = [ 'BandageVSSPAligner', 'ResultsBandageVSSPAligner','BandageVSGraphAligner',
       'ResultsBandageVSGraphAligner']

In [48]:
final_df = final_df.drop(columns=columns_to_drop)

In [49]:
final_df.to_csv('Final_df_categrories.tsv',sep='\t')